# Model Training

The work in this notebook focuses on the problem formulation and model selection, training and evaluation.

## Problem Formulation

In this section I work out the sliding window and prediction horizon. Since I couldn't find any mentions of recording frequency for this dataset, I will work with the assumption that the data is recorded in 1-minute intervals. The exact unknown time interval is not critical, this interpretation mainly improves readability of the experiment.

Because we work with multiple machines, all processing is done per machine first and only then the resulting samples are combined. This avoids mixing timelines and prevents look-ahead leakage from overlapping windows around the split point.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json

from pathlib import Path

In [8]:
DATA_DIR = Path("../data/processed/smd_group1")

csv_paths = sorted(DATA_DIR.glob("machine-1-*.csv"))
if not csv_paths:
    raise FileNotFoundError(
        f"No machine-1-*.csv files found under: {DATA_DIR.resolve()}"
    )

machines = {}
for p in csv_paths:
    df = pd.read_csv(p)

    if "label" not in df.columns:
        raise ValueError(f"Missing 'label' column in: {p.name}")

    # Ensure labels are ints (0/1)
    df["label"] = df["label"].astype(int)

    machines[p.stem] = df

print(f"Loaded machines: {len(machines)}")
print("Names:", list(machines.keys()))

Loaded machines: 8
Names: ['machine-1-1', 'machine-1-2', 'machine-1-3', 'machine-1-4', 'machine-1-5', 'machine-1-6', 'machine-1-7', 'machine-1-8']


In [9]:
# Verify consistent feature columns across machines
feature_cols_per_machine = {
    name: [c for c in df.columns if c != "label"] for name, df in machines.items()
}

first_machine = next(iter(machines.keys()))
base_features = feature_cols_per_machine[first_machine]

mismatched = {
    name: cols
    for name, cols in feature_cols_per_machine.items()
    if cols != base_features
}

print(f"Feature count (expected): {len(base_features)}")
if mismatched:
    print("WARNING: Feature columns mismatch detected:")
    for name, cols in mismatched.items():
        print(name, "->", len(cols))
else:
    print("All machines have consistent feature columns.")

Feature count (expected): 36
All machines have consistent feature columns.


### Prediction Horizon H

Determining the size of H presents a trade-off:
- Large (e.g. 60 min): great for operators, because they get a lot of time to fix the issue, but prediction accuracy typically drops. Predicting a crash 1 hour in advance from minute-level metrics is very hard.
- Small (e.g. 1 min): higher prediction accuracy, but operationally less useful. By the time the alert fires, the system may already be in a bad state.

For this project, I set H = 5 minutes. This corresponds to a realistic reaction window and allows me to evaluate detection lead time in a meaningful way.

### Sliding Window W

This parameter also presents a trade-off:

- Small W (e.g. 1–2 min): the model mainly sees the current spike and cannot detect longer trends.
- Large W (e.g. 60+ min): the model sees a lot of history, but the signal can be diluted with noise and irrelevant regime changes.

Cloud infrastructure failures can appear abruptly (shock failures) or build up over 10–20 minutes. Selecting an exact value is not obvious, so I will compare two window sizes:

- W = 12 minutes: captures short-term dynamics while remaining relatively noise-resistant.
- W = 25 minutes: captures longer build-ups, but may include more irrelevant variation.

Multi-machine processing:
For each machine, I slide a window of size W over the timeline, compute rolling statistics (Mean, Std, Delta, Z-Score), and define the target label based on whether an incident occurs within the next H steps. Then, for each machine, I split samples chronologically into train/test using a gap of (W + H) steps around the split point to prevent overlap leakage.

In [10]:
# Converts a single machine timeline into supervised samples
def create_supervised_dataset(df: pd.DataFrame, window_size: int, horizon: int):
    
    if "label" not in df.columns:
        raise ValueError("Expected 'label' column in df")

    feature_df = df.drop(columns=["label"])
    labels = df["label"].astype(int).to_numpy()
    X_raw = feature_df.to_numpy()

    X_list = []
    y_list = []
    t_end_list = []

    for t in range(window_size, len(X_raw) - horizon):
        window = X_raw[t - window_size : t]

        w_mean = np.mean(window, axis=0)
        w_std = np.std(window, axis=0)
        w_delta = window[-1] - window[-2]

        with np.errstate(divide="ignore", invalid="ignore"):
            w_zscore = (window[-1] - w_mean) / w_std
        w_zscore = np.nan_to_num(w_zscore, nan=0.0, posinf=0.0, neginf=0.0)

        features_vector = np.concatenate([w_mean, w_std, w_delta, w_zscore])

        future_window = labels[t : t + horizon]
        incident_soon = 1 if future_window.sum() > 0 else 0

        X_list.append(features_vector)
        y_list.append(incident_soon)
        t_end_list.append(t)

    return np.array(X_list), np.array(y_list), np.array(t_end_list)

In [11]:
def build_pooled_train_test(
    machines_dict: dict[str, pd.DataFrame],
    window_size: int,
    horizon: int,
    train_frac: float = 0.70,
):

    gap = window_size + horizon

    X_train_list, y_train_list = [], []
    X_test_list, y_test_list = [], []

    per_machine_stats = []

    for name, df in machines_dict.items():
        
        # Create supervised windows
        X, y, t_end = create_supervised_dataset(
            df=df, window_size=window_size, horizon=horizon
        )

        # Split chronologically
        split_t = int(len(df) * train_frac)

        # Apply gap
        train_mask = t_end < split_t
        test_mask = t_end >= split_t + gap

        X_train_m, y_train_m = X[train_mask], y[train_mask]
        X_test_m, y_test_m = X[test_mask], y[test_mask]

        if len(X_train_m) == 0 or len(X_test_m) == 0:
            raise ValueError(
                f"Empty split for {name} (W={window_size}, H={horizon}). "
                f"Try adjusting train_frac or verify series length."
            )

        X_train_list.append(X_train_m)
        y_train_list.append(y_train_m)
        X_test_list.append(X_test_m)
        y_test_list.append(y_test_m)

        per_machine_stats.append(
            {
                "machine": name,
                "train_windows": int(len(y_train_m)),
                "train_pos_rate": float(y_train_m.mean()),
                "test_windows": int(len(y_test_m)),
                "test_pos_rate": float(y_test_m.mean()),
            }
        )

    # Pool all train and test windows
    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test = np.concatenate(X_test_list, axis=0)
    y_test = np.concatenate(y_test_list, axis=0)

    stats_df = pd.DataFrame(per_machine_stats).sort_values("machine").reset_index(
        drop=True
    )

    return X_train, y_train, X_test, y_test, stats_df

Next, I create the two proposed datasets with the two values for W. 

In [12]:
H = 5
W_short = 12
W_long = 25

print("--- Creating pooled train/test sets (per-machine split with gap) ---")

print(f"\n[W={W_short}, H={H}]")
X_train_short, y_train_short, X_test_short, y_test_short, stats_short = (
    build_pooled_train_test(
        machines_dict=machines,
        window_size=W_short,
        horizon=H,
        train_frac=0.70,
    )
)

print(f"\n[W={W_long}, H={H}]")
X_train_long, y_train_long, X_test_long, y_test_long, stats_long = build_pooled_train_test(
    machines_dict=machines,
    window_size=W_long,
    horizon=H,
    train_frac=0.70,
)

print("\n--- Shape Verification ---")
print(f"Short Window (W=12): X_train={X_train_short.shape}, X_test={X_test_short.shape}")
print(f"Long Window  (W=25): X_train={X_train_long.shape}, X_test={X_test_long.shape}")

print("\n--- Class Balance post-windowing ---")
print(
    f"W=12: train_pos_rate={y_train_short.mean():.4f}, "
    f"test_pos_rate={y_test_short.mean():.4f}"
)
print(
    f"W=25: train_pos_rate={y_train_long.mean():.4f}, "
    f"test_pos_rate={y_test_long.mean():.4f}"
)

print("\n--- Per-machine window stats (W=12) ---")
display(stats_short)

print("\n--- Per-machine window stats (W=25) ---")
display(stats_long)

--- Creating pooled train/test sets (per-machine split with gap) ---

[W=12, H=5]

[W=25, H=5]

--- Shape Verification ---
Short Window (W=12): X_train=(135962, 144), X_test=(58140, 144)
Long Window  (W=25): X_train=(135858, 144), X_test=(58036, 144)

--- Class Balance post-windowing ---
W=12: train_pos_rate=0.0518, test_pos_rate=0.0883
W=25: train_pos_rate=0.0518, test_pos_rate=0.0883

--- Per-machine window stats (W=12) ---


,machine,train_windows,train_pos_rate,test_windows,test_pos_rate
0,machine-1-1,19923,0.107464,8522,0.066651
1,machine-1-2,16573,0.016895,7087,0.042613
2,machine-1-3,16580,0.027744,7089,0.057131
3,machine-1-4,16582,0.024665,7091,0.050628
4,machine-1-5,16582,0.006272,7090,0.003385
5,machine-1-6,16570,0.037236,7085,0.453211
6,machine-1-7,16575,0.142504,7088,0.012415
7,machine-1-8,16577,0.040116,7088,0.025113



--- Per-machine window stats (W=25) ---


,machine,train_windows,train_pos_rate,test_windows,test_pos_rate
0,machine-1-1,19910,0.107534,8509,0.065225
1,machine-1-2,16560,0.016908,7074,0.042692
2,machine-1-3,16567,0.027766,7076,0.057236
3,machine-1-4,16569,0.024685,7078,0.050721
4,machine-1-5,16569,0.006277,7077,0.003391
5,machine-1-6,16557,0.037265,7072,0.454044
6,machine-1-7,16562,0.142616,7075,0.012438
7,machine-1-8,16564,0.040147,7075,0.025159


## Model Selection and Training

Based on information observed during EDA as well as the nature of the problem, I have chosen a
Random Forest Classifier for this task.

The model will be trained on pooled sliding-window samples generated from all machines in SMD Group 1,
using a per-machine chronological split with a gap to prevent overlap leakage. This setup
matches the intended production workflow: the model is trained on historical data and evaluated on
strictly future time windows.

The main reasons for selecting Random Forest are:

1. **Non-Linear features:** In the bivariate analysis, I observed that some metrics are sparse / event-driven and spike during incidents. Linear models (e.g. Logistic Regression) struggle with such sharp thresholds, while decision trees capture them naturally.
2. **Interaction Effects:** Some features (like feat_15 or feat_22) appeared stable but are likely informative in combination. Random Forests can learn these interactions without manual feature engineering.
3. **Robustness to Noise:** Cloud metrics are noisy and non-stationary. By averaging the predictions of multiple trees, the Random Forest reduces the variance and is less likely to overfit to individual noisy data points compared to a single Decision Tree.
4. **Efficiency for Retraining:** The project description requires a system capable of periodic retraining. Random Forests train quickly, parallelize well, and are feasible for periodic retraining in a lightweight deployment setup.


### Training Strategy

I will train two separate models to compare the impact of the window size:
1. Model A: Trained on \(W=12\)
2. Model B: Trained on \(W=25\)

After converting the series into supervised sliding-window samples, the resulting class balance is imbalanced and can vary by machine. To ensure the model focuses on capturing incidents (Recall) rather than maximizing accuracy, I'll use `class_weight="balanced"` or `"balanced_subsample"`. This penalizes missed incidents more than false alarms.

To prevent look-ahead leakage, all splits are chronological per machine and include a purge gap of (W + H) around the split point.

Goal: Following the project description, I target approximately 80% recall with respect to existing incident-triggering alerts on a held-out evaluation period. The alert threshold will be selected on a validation period and then fixed for the final evaluation.

### Hyperparameter Tuning Strategy - Randomized Search

Because sliding-window samples overlap heavily in time, many training rows are near-duplicates. This increases the risk that a high-capacity model memorizes recurring patterns instead of actual pre-incident signals. Additionally, metric behavior can shift over time, so regularization and robustness matter more than marginal in-sample accuracy.

Instead of an exhaustive grid search, I will use Randomized Search over a carefully chosen parameter
space. Randomized search is more compute-efficient and typically finds strong configurations with
fewer fits, especially when only a subset of hyperparameters strongly influences performance.

I tune the following parameters (highest impact first):

- **min_samples_leaf:** Increases the minimum support required for a decision rule. This is a key regularizer for overlapping windows and noisy spikes. Larger values reduce overfitting and improve stability.
- **max_depth:** Limits tree complexity. Prevents very deep trees from learning too specific rules that do not generalize to future periods or other machines.
- **max_features:** Controls how many features are considered at each split. Lower values decorrelate trees and generally improve generalization in noisy settings.
- **max_samples (with bootstrap=True):** Subsamples training rows for each tree, reducing correlation between trees and improving robustness. Useful when training data contains many overlapping windows.
- **n_estimators:** Controls ensemble size. More trees reduce variance and stabilize probabilities at the cost of training time. I search over moderate-to-high values.
- **class_weight:** Incident-imminent samples are rare after windowing. class_weight="balanced_subsample" adjusts the class weighting per tree bootstrap sample and helps the model pay attention to positives.

Tuning protocol:
- Splits are chronological per machine and include a purge gap of (W + H) steps to avoid overlap
  leakage across split boundaries.
- Hyperparameters are selected using a time-consistent validation set (future relative to the
  sub-train period).
- The tuning score is Average Precision (PR-AUC), which is suitable for imbalanced classification.
  The final alert threshold selection (to target ~80% recall) is handled later during evaluation.

In [13]:
import math

from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from sklearn.model_selection import ParameterSampler
from joblib import dump

In [14]:
def build_pooled_train_val_test(
    machines_dict: dict[str, pd.DataFrame],
    window_size: int,
    horizon: int,
    train_frac: float = 0.70,
    val_frac_within_train: float = 0.20,
    dtype=np.float32,
):
    gap = window_size + horizon

    X_tr_list, y_tr_list = [], []
    X_val_list, y_val_list = [], []
    X_te_list, y_te_list = [], []

    stats = []

    for name, df in machines_dict.items():
        X, y, t_end = create_supervised_dataset(
            df=df, window_size=window_size, horizon=horizon
        )

        n = len(df)
        cut_train_end = int(n * train_frac)
        cut_val_start = int(cut_train_end * (1 - val_frac_within_train))

        tr_mask = t_end < cut_val_start
        val_mask = (t_end >= cut_val_start + gap) & (t_end < cut_train_end)
        te_mask = t_end >= cut_train_end + gap

        X_tr_m, y_tr_m = X[tr_mask], y[tr_mask]
        X_val_m, y_val_m = X[val_mask], y[val_mask]
        X_te_m, y_te_m = X[te_mask], y[te_mask]

        if len(X_tr_m) == 0 or len(X_val_m) == 0 or len(X_te_m) == 0:
            raise ValueError(
                f"Empty split for {name}. "
                f"W={window_size}, H={horizon}, "
                f"len(tr)={len(X_tr_m)}, len(val)={len(X_val_m)}, len(te)={len(X_te_m)}. "
                "Adjust fractions or verify series length."
            )

        X_tr_list.append(X_tr_m.astype(dtype, copy=False))
        y_tr_list.append(y_tr_m.astype(np.int8, copy=False))
        X_val_list.append(X_val_m.astype(dtype, copy=False))
        y_val_list.append(y_val_m.astype(np.int8, copy=False))
        X_te_list.append(X_te_m.astype(dtype, copy=False))
        y_te_list.append(y_te_m.astype(np.int8, copy=False))

        stats.append(
            {
                "machine": name,
                "train_windows": int(len(y_tr_m)),
                "train_pos_rate": float(y_tr_m.mean()),
                "val_windows": int(len(y_val_m)),
                "val_pos_rate": float(y_val_m.mean()),
                "test_windows": int(len(y_te_m)),
                "test_pos_rate": float(y_te_m.mean()),
            }
        )

    X_tr = np.concatenate(X_tr_list, axis=0)
    y_tr = np.concatenate(y_tr_list, axis=0)
    X_val = np.concatenate(X_val_list, axis=0)
    y_val = np.concatenate(y_val_list, axis=0)
    X_te = np.concatenate(X_te_list, axis=0)
    y_te = np.concatenate(y_te_list, axis=0)

    stats_df = pd.DataFrame(stats).sort_values("machine").reset_index(drop=True)
    return X_tr, y_tr, X_val, y_val, X_te, y_te, stats_df

For each machine timeline, this function:
- Creates supervised windows
- Splits into sub-train / validation parts within the training period
- Splits into test on the held-out future period
- Applies a purge gap W+H around split points to prevent overlap leakage
- Returns pooled arrays across machines

In [15]:
def randomized_search_rf(
    X_tr,
    y_tr,
    X_val,
    y_val,
    param_distributions: dict,
    n_iter: int = 40,
    random_state: int = 42,
    n_jobs: int = -1,
    verbose: int = 1,
):
    sampler = ParameterSampler(
        param_distributions=param_distributions,
        n_iter=n_iter,
        random_state=random_state,
    )

    history = []
    best_score = -np.inf
    best_model = None
    best_params = None

    for i, params in enumerate(sampler, start=1):
        model = RandomForestClassifier(
            random_state=random_state,
            n_jobs=n_jobs,
            **params,
        )
        model.fit(X_tr, y_tr)

        proba = model.predict_proba(X_val)[:, 1]
        ap = average_precision_score(y_val, proba)

        row = {"iter": i, "avg_precision": float(ap), **params}
        history.append(row)

        if verbose:
            print(f"[{i:02d}/{n_iter}] AP={ap:.5f} params={params}")

        if ap > best_score:
            best_score = ap
            best_model = model
            best_params = params

    history_df = pd.DataFrame(history).sort_values(
        "avg_precision", ascending=False
    ).reset_index(drop=True)

    return best_model, best_params, history_df

This function implements randomized search for sample hyperparameters and scores on (X_val, y_val) using Average Precision (PR-AUC). At the end, it returns the best_estimator, best_params and history_df.

In [16]:
param_distributions = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 6, 10, 14, 18],
    "min_samples_leaf": [5, 10, 20],
    "min_samples_split": [5, 10, 20],
    "max_features": ["sqrt", 0.3, 0.5, 0.7],
    "bootstrap": [True],
    "max_samples": [0.5, 0.7, 0.9, 1.0],
    "class_weight": ["balanced_subsample"],
}

This is the hyperparameter space to search based on definition above.

In [17]:
# Extensive search that runs for a long time, for a demo, lower the n_iter parameter to 2-5

# OUT_DIR = Path("../artifacts/rf_random_search_group1")
# OUT_DIR.mkdir(parents=True, exist_ok=True)

# H = 5
# W_values = [12, 25]

# results = {}

# for W in W_values:
#     print("\n" + "=" * 80)
#     print(f"TUNING RANDOM FOREST | W={W}, H={H}")
#     print("=" * 80)

#     X_tr, y_tr, X_val, y_val, X_te, y_te, stats_df = build_pooled_train_val_test(
#         machines_dict=machines,
#         window_size=W,
#         horizon=H,
#         train_frac=0.70,
#         val_frac_within_train=0.20,
#     )

#     print("\nSplit summary (per machine):")
#     display(stats_df)

#     print("\nPooled shapes:")
#     print(f"X_tr={X_tr.shape}, y_tr_pos_rate={y_tr.mean():.4f}")
#     print(f"X_val={X_val.shape}, y_val_pos_rate={y_val.mean():.4f}")
#     print(f"X_te={X_te.shape}, y_te_pos_rate={y_te.mean():.4f}  (kept for later)")

#     best_model, best_params, hist_df = randomized_search_rf(
#         X_tr=X_tr,
#         y_tr=y_tr,
#         X_val=X_val,
#         y_val=y_val,
#         param_distributions=param_distributions,
#         n_iter=40,
#         random_state=42,
#         n_jobs=-1,
#         verbose=1,
#     )

#     print("\nBest params:")
#     print(best_params)
#     print("\nBest validation AP (PR-AUC):", hist_df.iloc[0]["avg_precision"])

#     # Save search history + params + model
#     hist_path = OUT_DIR / f"search_history_W{W}_H{H}.csv"
#     hist_df.to_csv(hist_path, index=False)

#     params_path = OUT_DIR / f"best_params_W{W}_H{H}.json"
#     with open(params_path, "w", encoding="utf-8") as f:
#         json.dump(best_params, f, indent=2)

#     model_path = OUT_DIR / f"best_model_W{W}_H{H}.joblib"
#     dump(best_model, model_path)

#     results[W] = {
#         "best_params": best_params,
#         "best_val_ap": float(hist_df.iloc[0]["avg_precision"]),
#         "history_path": str(hist_path),
#         "model_path": str(model_path),
#     }

# results


# I ran the search and saved the results
# Instead of repeating the search above, load the saved search history and analyze results:
def summarize_param_effects(hist_df: pd.DataFrame, params: list[str]):
    out = {}
    for p in params:
        out[p] = (
            hist_df.groupby(p)["avg_precision"]
            .agg(["count", "mean", "median", "std", "max"])
            .sort_values("mean", ascending=False)
        )
    return out


params_to_check = [
    "n_estimators",
    "max_depth",
    "min_samples_leaf",
    "min_samples_split",
    "max_features",
    "max_samples",
]

hist_df_12 = pd.read_csv("../artifacts/rf_random_search_group1/search_history_W12_H5.csv")
hist_df_25 = pd.read_csv("../artifacts/rf_random_search_group1/search_history_W25_H5.csv")


summaries_12 = summarize_param_effects(hist_df_12, params_to_check)
print("-------------------------------------------------- W=12 --------------------------------------------------")
for p, table in summaries_12.items():
    print("\n" + "=" * 60)
    print(p)
    display(table)


summaries_25 = summarize_param_effects(hist_df_25, params_to_check)

print("-------------------------------------------------- W=25 --------------------------------------------------")
for p, table in summaries_25.items():
    print("\n" + "=" * 60)
    print(p)
    display(table)

-------------------------------------------------- W=12 --------------------------------------------------

n_estimators


,count,mean,median,std,max
n_estimators,,,,,
100,14,0.598027,0.602285,0.030357,0.647160
200,14,0.575987,0.590493,0.069536,0.616955
300,12,0.568338,0.582558,0.045549,0.610600



max_depth


,count,mean,median,std,max
max_depth,,,,,
14.0,8,0.612496,0.610134,0.019480,0.647160
18.0,10,0.596977,0.592612,0.016045,0.623512
10.0,8,0.563542,0.573758,0.048704,0.616955
6.0,6,0.511147,0.545033,0.084495,0.555844



min_samples_leaf


...

,count,mean,median,std,max
min_samples_split,,,,,
20,16,0.626048,0.640494,0.049562,0.674736
5,13,0.609928,0.614767,0.040132,0.653128
10,11,0.607586,0.628399,0.095887,0.662597



max_features


,count,mean,median,std,max
max_features,,,,,
0.7,13,0.652533,0.655560,0.014167,0.674736
0.5,9,0.636567,0.639936,0.016578,0.662597
0.3,11,0.613816,0.619557,0.022879,0.641052
sqrt,7,0.523609,0.574158,0.100155,0.602595



max_samples


,count,mean,median,std,max
max_samples,,,,,
0.5,11,0.636534,0.641052,0.027229,0.674736
0.7,12,0.621983,0.624154,0.025313,0.653128
1.0,10,0.615912,0.628362,0.047631,0.662127
0.9,7,0.572069,0.628777,0.127496,0.662597


### Random Search Findings 

I ran 40 iterations of the algorithm above on both W=12 and W=25. The search history as well as the best found models and their parameters can be found in the artifacts folder. Here are the key findings from this search:

Across both window sizes, similar trends emerged:

- Subsampling helps: max_samples=0.5 consistently achieved the best mean AP, while 0.9/1.0 tended to perform worse or be less stable. This matches the heavy overlap of sliding windows. Subsampling reduces redundancy per tree and improves ensemble diversity.

- Shallow trees underfit: max_depth=6 (and often 10) produced the lowest AP. Deeper trees performed better, suggesting the model needs sufficient depth to represent enough conditional logic.

- Feature subsampling matters: max_features="sqrt" was consistently the worst option. Fractional values performed better. 

- Stronger regularization is generally better: min_samples_split=20 and min_samples_leaf=10–20 were consistently strong choices, while very small values (5) appeared frequently among weaker runs.


Overall, W=25 achieved a slightly higher validation AP on average than W=12, suggesting that a longer lookback window captures additional pre-incident context for this dataset. To see if there is a statistically significant difference between the mean randomly sampled hyperparameter configurations for W=25 and W=12, I will next do a hypothesis test on the 40 AP values.

*Note on overfitting:*

Hyperparameters are selected on a time-consistent validation period, which reduces look-ahead bias. However, selecting the best configuration from many trials can still overfit to the validation period (“hyperparameter overfitting”). A more robust setup would evaluate candidates across multiple walk-forward validation splits or across held-out machines.

In [18]:
hist_df_12 = pd.read_csv(
    "../artifacts/rf_random_search_group1/search_history_W12_H5.csv"
)
hist_df_25 = pd.read_csv(
    "../artifacts/rf_random_search_group1/search_history_W25_H5.csv"
)

ap12 = hist_df_12["avg_precision"].to_numpy(dtype=float)
ap25 = hist_df_25["avg_precision"].to_numpy(dtype=float)

print("n12:", len(ap12), "mean12:", ap12.mean(), "std12:", ap12.std(ddof=1))
print("n25:", len(ap25), "mean25:", ap25.mean(), "std25:", ap25.std(ddof=1))
print("mean diff (25-12):", ap25.mean() - ap12.mean())

# Check if the two random searches used the exact same sampled params per iteration.
param_cols = [
    "n_estimators",
    "max_depth",
    "min_samples_leaf",
    "min_samples_split",
    "max_features",
    "bootstrap",
    "max_samples",
    "class_weight",
]

paired = False
if all(c in hist_df_12.columns for c in param_cols) and all(
    c in hist_df_25.columns for c in param_cols
):
    df12_params = hist_df_12.sort_values("iter")[param_cols].reset_index(drop=True)
    df25_params = hist_df_25.sort_values("iter")[param_cols].reset_index(drop=True)
    paired = df12_params.equals(df25_params)

print("Paired by identical hyperparameter samples:", paired)

alpha = 0.05

# 1) Parametric test
if paired:
    # Paired t-test on per-iteration AP differences
    ap12_sorted = hist_df_12.sort_values("iter")["avg_precision"].to_numpy(dtype=float)
    ap25_sorted = hist_df_25.sort_values("iter")["avg_precision"].to_numpy(dtype=float)
    diffs = ap25_sorted - ap12_sorted

    # scipy doesn't always support alternative for ttest_rel on older versions,
    # so compute one-sided p-value manually from two-sided
    t_stat, p_two = stats.ttest_rel(ap25_sorted, ap12_sorted, nan_policy="raise")
    p_one = p_two / 2 if t_stat > 0 else 1 - (p_two / 2)

    print("\nPaired t-test (H1: mean25 > mean12)")
    print("t =", t_stat, "p(one-sided) =", p_one)

    # Effect size for paired differences: Cohen's dz
    dz = diffs.mean() / diffs.std(ddof=1)
    print("Effect size (Cohen's dz) =", dz)
else:
    # Welch's t-test (unequal variances)
    # Use alternative="greater" if available, else manual one-sided conversion.
    try:
        res = stats.ttest_ind(ap25, ap12, equal_var=False, alternative="greater")
        t_stat, p_one = float(res.statistic), float(res.pvalue)
    except TypeError:
        t_stat, p_two = stats.ttest_ind(ap25, ap12, equal_var=False)
        p_one = p_two / 2 if t_stat > 0 else 1 - (p_two / 2)

    print("\nWelch t-test (unpaired) (H1: mean25 > mean12)")
    print("t =", t_stat, "p(one-sided) =", p_one)

    # Effect size (Cohen's d with pooled SD; descriptive)
    s12 = ap12.std(ddof=1)
    s25 = ap25.std(ddof=1)
    s_pooled = np.sqrt((s12**2 + s25**2) / 2)
    d = (ap25.mean() - ap12.mean()) / s_pooled
    print("Effect size (Cohen's d, descriptive) =", d)

# 2) Permutation test on mean difference (robust, minimal assumptions)
rng = np.random.default_rng(2026)
observed = ap25.mean() - ap12.mean()

combined = np.concatenate([ap25, ap12])
n25 = len(ap25)

n_perm = 50_000
count = 0
for _ in range(n_perm):
    rng.shuffle(combined)
    perm_25 = combined[:n25]
    perm_12 = combined[n25:]
    perm_diff = perm_25.mean() - perm_12.mean()
    if perm_diff >= observed:
        count += 1

p_perm = (count + 1) / (n_perm + 1)

print("\nPermutation test (H1: mean25 > mean12)")
print("observed mean diff =", observed)
print("p(one-sided) =", p_perm)

print("\nDecision at alpha =", alpha)
print("Reject H0 (perm test):", p_perm < alpha)

n12: 40 mean12: 0.5814060080922664 std12: 0.0516366819279887
n25: 40 mean25: 0.615731863576408 std25: 0.062221127452515367
mean diff (25-12): 0.034325855484141576
Paired by identical hyperparameter samples: True

Paired t-test (H1: mean25 > mean12)
t = 7.237272080355264 p(one-sided) = 5.072294793991226e-09
Effect size (Cohen's dz) = 1.144313191013389

Permutation test (H1: mean25 > mean12)
observed mean diff = 0.034325855484141576
p(one-sided) = 0.0030599388012239755

Decision at alpha = 0.05
Reject H0 (perm test): True


The two searches were paired: each iteration evaluated the same randomly sampled RF
hyperparameters under both window sizes (only W differed). This allows a paired hypothesis test:

- H0: mean(AP_W12) = mean(AP_W25)
- H1: mean(AP_W25) > mean(AP_W12)

Results (n=40 paired runs):
- mean(AP_W12) = 0.5814 (std = 0.0516)
- mean(AP_W25) = 0.6157 (std = 0.0622)
- mean difference = +0.0343 AP in favor of W=25

Paired t-test (one-sided): p = 5.07e-09
Permutation test (one-sided): p = 0.00306
Effect size (Cohen’s dz) = 1.14

Conclusion: W=25 yields a statistically significant and practically meaningful improvement in
validation AP over W=12. Therefore, moving forward, I will now focus on tuning for W=25.

In [19]:
def successive_halving_rf(
    X_tr,
    y_tr,
    X_val,
    y_val,
    param_distributions: dict,
    n_candidates: int = 30,
    n_estimators_schedule=(50, 150, 400),
    keep_ratio: float = 1 / 3,
    random_state: int = 42,
    n_jobs: int = -1,
):
    sampler = ParameterSampler(
        param_distributions=param_distributions,
        n_iter=n_candidates,
        random_state=random_state,
    )
    candidates = list(sampler)

    all_rows = []
    survivors = candidates

    best_params = None
    best_n_estimators = None
    best_ap = float("-inf")

    drop_cols = {"avg_precision", "stage", "n_estimators"}

    for stage, n_estimators in enumerate(n_estimators_schedule, start=1):
        print(f"Stage {stage}...")
        stage_rows = []

        for i, params in enumerate(survivors, start=1):
            print(f"Fitting model {i}...")
            model = RandomForestClassifier(
                n_estimators=n_estimators,
                random_state=random_state,
                n_jobs=n_jobs,
                **params,
            )
            model.fit(X_tr, y_tr)
            proba = model.predict_proba(X_val)[:, 1]
            ap = average_precision_score(y_val, proba)

            row = {
                "stage": stage,
                "n_estimators": n_estimators,
                "avg_precision": float(ap),
                **params,
            }
            stage_rows.append(row)

        stage_rows_sorted = sorted(
            stage_rows, key=lambda r: r["avg_precision"], reverse=True
        )

        if stage_rows_sorted[0]["avg_precision"] > best_ap:
            best_ap = float(stage_rows_sorted[0]["avg_precision"])
            best_n_estimators = int(stage_rows_sorted[0]["n_estimators"])
            best_params = {
                k: v for k, v in stage_rows_sorted[0].items() if k not in drop_cols
            }

        stage_df = pd.DataFrame(stage_rows_sorted)
        all_rows.append(stage_df)

        k = max(1, math.ceil(len(stage_rows_sorted) * keep_ratio))
        survivors = [
            {kk: vv for kk, vv in r.items() if kk not in drop_cols}
            for r in stage_rows_sorted[:k]
        ]

        print(
            f"Stage {stage}: n_estimators={n_estimators}, "
            f"candidates={len(stage_rows_sorted)} -> keep={k}, "
            f"best AP={stage_rows_sorted[0]['avg_precision']:.5f}"
        )

    history_df = pd.concat(all_rows, axis=0, ignore_index=True)
    return best_params, best_n_estimators, best_ap, history_df

In [23]:
# Successive search implementation, takes a while to compute.

# W = 25
# H = 5

# X_tr, y_tr, X_val, y_val, X_te, y_te, _ = build_pooled_train_val_test(
#     machines_dict=machines,
#     window_size=W,
#     horizon=H,
#     train_frac=0.70,
#     val_frac_within_train=0.20,
# )

# param_space_w25 = {
#     "max_depth": [14, 18, None],
#     "min_samples_leaf": [10, 20],
#     "min_samples_split": [20, 50, 100],
#     "max_features": [0.5, 0.7],
#     "bootstrap": [True],
#     "max_samples": [0.5, 0.7],
#     "class_weight": ["balanced_subsample"],
# }

# best_params, best_n_est, best_ap, sh_hist = successive_halving_rf(
#     X_tr=X_tr,
#     y_tr=y_tr,
#     X_val=X_val,
#     y_val=y_val,
#     param_distributions=param_space_w25,
#     n_candidates=30,
#     n_estimators_schedule=(50, 150, 400),
#     keep_ratio=1 / 3,
#     random_state=2026,
#     n_jobs=-1,
# )

# print("\nBest found:")
# print("AP:", best_ap)
# print("n_estimators:", best_n_est)
# print("params:", best_params)

# display(sh_hist.sort_values("avg_precision", ascending=False).head(10))


# I ran the search and saved the results
# Instead of repeating the search above, load the saved search history and analyze results:

def load_search_history(out_dir: str | Path, run_name: str) -> pd.DataFrame:
    out_dir = Path(out_dir)
    parquet_path = out_dir / f"{run_name}.parquet"
    csv_path = out_dir / f"{run_name}.csv"

    if parquet_path.exists():
        df = pd.read_parquet(parquet_path)
    elif csv_path.exists():
        df = pd.read_csv(csv_path)
    else:
        raise FileNotFoundError(f"Missing {parquet_path} and {csv_path}")

    if "max_depth" in df.columns:
        df["max_depth"] = df["max_depth"].where(
            ~df["max_depth"].isna(), other=None
        )

    return df


def show_top_results(history_df: pd.DataFrame, top_n: int = 10):
    cols = [
        c
        for c in [
            "stage",
            "n_estimators",
            "avg_precision",
            "max_depth",
            "min_samples_leaf",
            "min_samples_split",
            "max_features",
            "max_samples",
            "class_weight",
            "bootstrap",
        ]
        if c in history_df.columns
    ]
    top = history_df.sort_values("avg_precision", ascending=False).head(top_n)
    display(top[cols])

    if "stage" in history_df.columns:
        print("\nBest AP by stage:")
        display(
            history_df.groupby("stage")["avg_precision"]
            .max()
            .rename("best_avg_precision")
            .reset_index()
        )


OUT_DIR: str | Path = "../artifacts/rf_successive_halving_group1"
run_name: str = "successive_halving_W25_H5_c30_sched50-150-400"
sh_hist_loaded = load_search_history(OUT_DIR, run_name)
show_top_results(sh_hist_loaded, top_n=10)

,stage,n_estimators,avg_precision,max_depth,min_samples_leaf,min_samples_split,max_features,max_samples,class_weight,bootstrap
40,3,400,0.668961,14.0,10,20,0.7,0.5,balanced_subsample,True
41,3,400,0.667434,14.0,20,50,0.7,0.5,balanced_subsample,True
30,2,150,0.664330,NaN,20,50,0.7,0.5,balanced_subsample,True
31,2,150,0.663952,18.0,20,20,0.7,0.5,balanced_subsample,True
42,3,400,0.663724,18.0,20,20,0.7,0.5,balanced_subsample,True
32,2,150,0.662185,14.0,20,50,0.7,0.5,balanced_subsample,True
43,3,400,0.660921,NaN,20,50,0.7,0.5,balanced_subsample,True
0,1,50,0.660824,14.0,20,50,0.7,0.5,balanced_subsample,True
33,2,150,0.659616,14.0,10,20,0.7,0.5,balanced_subsample,True
1,1,50,0.658833,NaN,20,50,0.7,0.5,balanced_subsample,True



Best AP by stage:


,stage,best_avg_precision
0,1,0.660824
1,2,0.664330
2,3,0.668961


After an initial broad random search and a refined search, further optimization produces only minimal changes in validation AP. I will choose the final model from the top configurations based on which one has the best scores over multiple different random states (seeds).

In [24]:
def ap_over_seeds(
    params: dict,
    n_estimators: int,
    seeds: list[int],
    X_tr,
    y_tr,
    X_val,
    y_val,
):
    aps = []
    for s in seeds:
        print(f"Fitting seed {s}")
        model = RandomForestClassifier(
            **params,
            n_estimators=n_estimators,
            random_state=s,
            n_jobs=-1,
        )
        model.fit(X_tr, y_tr)
        ap = average_precision_score(y_val, model.predict_proba(X_val)[:, 1])
        aps.append(float(ap))

    return float(np.mean(aps)), float(np.std(aps)), aps

In [25]:
# Seed robustness check implementation, takes a while to compute.

# def row_to_params(row: pd.Series) -> tuple[dict, int]:
#     n_estimators = int(row["n_estimators"])

#     max_depth = row.get("max_depth", None)
#     if pd.isna(max_depth):
#         max_depth = None
#     else:
#         max_depth = int(max_depth)

#     params = {
#         "max_depth": max_depth,
#         "min_samples_leaf": int(row["min_samples_leaf"]),
#         "min_samples_split": int(row["min_samples_split"]),
#         "max_features": row["max_features"],
#         "bootstrap": bool(row["bootstrap"]),
#         "max_samples": float(row["max_samples"]),
#         "class_weight": row["class_weight"],
#     }

#     if isinstance(params["max_features"], str) and params["max_features"] != "sqrt":
#         params["max_features"] = float(params["max_features"])

#     return params, n_estimators

# def select_best_by_seed_robustness(
#     history_df: pd.DataFrame,
#     X_tr,
#     y_tr,
#     X_val,
#     y_val,
#     top_k: int = 5,
#     seeds: list[int] | None = None,
#     stage: int | None = None,
# ):
#     if seeds is None:
#         seeds = [0, 1, 2, 3, 4]

#     df = history_df.copy()

#     if stage is not None and "stage" in df.columns:
#         df = df[df["stage"] == stage].copy()

#     df = df.sort_values("avg_precision", ascending=False).head(top_k)

#     rows = []
#     for _, r in df.iterrows():
#         params, n_estimators = row_to_params(r)
#         print(params)
#         mean_ap, std_ap, aps = ap_over_seeds(
#             params=params,
#             n_estimators=n_estimators,
#             seeds=seeds,
#             X_tr=X_tr,
#             y_tr=y_tr,
#             X_val=X_val,
#             y_val=y_val,
#         )
#         rows.append(
#             {
#                 "orig_avg_precision": float(r["avg_precision"]),
#                 "n_estimators": n_estimators,
#                 "mean_ap": mean_ap,
#                 "std_ap": std_ap,
#                 "aps": aps,
#                 "params": params,
#             }
#         )

#     out = pd.DataFrame(rows).sort_values("mean_ap", ascending=False).reset_index(
#         drop=True
#     )
#     return out


# W = 25
# H = 5
# X_tr, y_tr, X_val, y_val, X_te, y_te, _ = build_pooled_train_val_test(
#     machines_dict=machines,
#     window_size=W,
#     horizon=H,
#     train_frac=0.70,
#     val_frac_within_train=0.20,
# )

# print("Validation positive rate:", y_val.mean())

# robust_df = select_best_by_seed_robustness(
#     history_df=sh_hist,  # from successive halving
#     X_tr=X_tr,
#     y_tr=y_tr,
#     X_val=X_val,
#     y_val=y_val,
#     top_k=5,
#     seeds=[0, 10, 100, 500, 2026],
#     stage=3,  # final stage
# )

# display(robust_df[["orig_avg_precision", "n_estimators", "mean_ap", "std_ap", "aps"]])
# best_choice = robust_df.iloc[0]
# best_choice["params"], best_choice["n_estimators"], best_choice["mean_ap"]


# I ran the search and saved the results
# Instead of repeating the search above, load the saved search history and analyze results:
def load_robustness_df(
    out_dir: str | Path = "../artifacts/rf_seed_check_group1",
    run_name: str = "seed_robustness_W25_H5",
) -> pd.DataFrame:
    out_dir = Path(out_dir)
    parquet_path = out_dir / f"{run_name}.parquet"
    csv_path = out_dir / f"{run_name}.csv"

    if parquet_path.exists():
        df = pd.read_parquet(parquet_path)
    elif csv_path.exists():
        df = pd.read_csv(csv_path)
    else:
        raise FileNotFoundError(f"Missing {parquet_path} and {csv_path}")

    if "params_json" in df.columns:
        df["params"] = df["params_json"].apply(
            lambda s: json.loads(s) if pd.notna(s) else None
        )
        df = df.drop(columns=["params_json"])

    if "aps_json" in df.columns:
        df["aps"] = df["aps_json"].apply(
            lambda s: json.loads(s) if pd.notna(s) else None
        )
        df = df.drop(columns=["aps_json"])

    return df


def print_robustness_summary(df: pd.DataFrame):
    cols = [c for c in ["orig_avg_precision", "n_estimators", "mean_ap", "std_ap"] if c in df.columns]
    display(df.sort_values("mean_ap", ascending=False)[cols])

    best = df.sort_values("mean_ap", ascending=False).iloc[0]
    print("\nBest by mean_ap:")
    print({k: best[k] for k in cols if k in best.index})
    if "params" in df.columns:
        print("params:", best["params"])

robust_df_loaded = load_robustness_df(
    out_dir="../artifacts/rf_seed_check_group1",
    run_name="seed_robustness_W25_H5",
)

print_robustness_summary(robust_df_loaded)

,orig_avg_precision,n_estimators,mean_ap,std_ap
0,0.668961,400,0.669783,0.001635
1,0.663724,400,0.665585,0.003064
2,0.667434,400,0.659493,0.005404
3,0.660921,400,0.653307,0.006096



Best by mean_ap:
{'orig_avg_precision': np.float64(0.6689614422037214), 'n_estimators': np.int64(400), 'mean_ap': np.float64(0.6697833227880404), 'std_ap': np.float64(0.0016353255395577)}
params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 14, 'max_features': 0.7, 'max_samples': 0.5, 'min_samples_leaf': 10, 'min_samples_split': 20}


In [27]:
from joblib import dump
from sklearn.ensemble import RandomForestClassifier

# Use the chosen robust params from above
final_params = robust_df_loaded.sort_values("mean_ap", ascending=False).iloc[0]["params"]
n_estimators_tuned = int(robust_df_loaded.sort_values("mean_ap", ascending=False).iloc[0]["n_estimators"])

# Increase trees for final fit (stabilizes probabilities)
# Rule of thumb: 2x–5x the tuned value; keep compute reasonable
n_estimators_final = max(800, 2 * n_estimators_tuned)

W = 25
H = 5
X_tr, y_tr, X_val, y_val, X_te, y_te, _ = build_pooled_train_val_test(
    machines_dict=machines,
    window_size=W,
    horizon=H,
    train_frac=0.70,
    val_frac_within_train=0.20,
)

X_trainval = np.concatenate([X_tr, X_val], axis=0)
y_trainval = np.concatenate([y_tr, y_val], axis=0)

final_model = RandomForestClassifier(
    **final_params,
    n_estimators=n_estimators_final,
    random_state=2026,
    n_jobs=-1,
)
final_model.fit(X_trainval, y_trainval)

ART_DIR = Path("../artifacts/final_rf_group1")
ART_DIR.mkdir(parents=True, exist_ok=True)

model_path = ART_DIR / f"rf_final_W{W}_H{H}.joblib"
dump(final_model, model_path)

meta = {
    "W": W,
    "H": H,
    "train_frac": 0.70,
    "val_frac_within_train": 0.20,
    "purge_gap": W + H,
    "n_estimators_tuned": n_estimators_tuned,
    "n_estimators_final": n_estimators_final,
    "params": final_params,
    "val_pos_rate": float(y_val.mean()),
}

meta_path = ART_DIR / f"rf_final_W{W}_H{H}.json"
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)

print("Saved final model to:", model_path)
print("Saved metadata to:", meta_path)

Saved final model to: ../artifacts/final_rf_group1/rf_final_W25_H5.joblib
Saved metadata to: ../artifacts/final_rf_group1/rf_final_W25_H5.json


### Final Model Choice (W = 25) via Seed Robustness Check

To reduce the risk of selecting a configuration that is “lucky” due to Random Forest randomness
(bootstrapping / feature subsampling), I evaluated the top candidate configurations across multiple
random seeds on the validation set.

The best candidate achieved mean validation AP ≈ 0.669 with std ≈ 0.001 across 5 seeds, indicating
high stability. I select this configuration as the final W=25 model and proceed to refit it on the
combined train+validation data before final evaluation on the held-out test period.